<a href="https://colab.research.google.com/github/annie-lyu/word-embedding/blob/master/Copy_of_ldaseq_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from gensim import corpora
from gensim.models import ldaseqmodel
from gensim.corpora import Dictionary, bleicorpus
import numpy
from gensim.matutils import hellinger
import nltk


In [2]:
import glob
from google.colab import drive
drive.mount('/content/drive')
##folder lists
ls = !ls "/content/drive/My Drive/words/sample"
ls = str(ls).strip("'[")
ls = ls.strip("]'")
entries = ls.split("\\t")
entries

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


['month1', 'month2', 'month3']

In [3]:
## preparation for text preprocessing
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = list(set(stopwords.words('english'))) + ["us", "also", "said", "would", "could", "mr", "get"]
from nltk.stem import PorterStemmer
ps = PorterStemmer()
stop_words

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


['him',
 'mightn',
 "should've",
 'some',
 'because',
 'each',
 't',
 'has',
 'did',
 'more',
 'same',
 "aren't",
 'ma',
 'under',
 'while',
 'have',
 'down',
 'over',
 'to',
 're',
 'wouldn',
 'if',
 'how',
 'll',
 'weren',
 'only',
 "you'll",
 'that',
 'haven',
 'there',
 'o',
 'doesn',
 'yours',
 'was',
 'being',
 'he',
 'and',
 'just',
 'than',
 'they',
 'aren',
 'isn',
 'own',
 'so',
 'does',
 'both',
 'now',
 'through',
 'myself',
 'doing',
 'an',
 "needn't",
 'hadn',
 "it's",
 'before',
 "won't",
 've',
 'hasn',
 'my',
 "she's",
 'with',
 'their',
 'been',
 'off',
 'himself',
 'theirs',
 'these',
 'between',
 'yourselves',
 'her',
 'wasn',
 'until',
 'in',
 'but',
 'ain',
 "weren't",
 'out',
 "shouldn't",
 'into',
 'up',
 "hadn't",
 'such',
 'it',
 'when',
 'm',
 'as',
 'y',
 'this',
 "shan't",
 'about',
 'itself',
 'are',
 'where',
 'd',
 'be',
 "wasn't",
 'above',
 'most',
 'had',
 'whom',
 'of',
 'after',
 'your',
 'for',
 'who',
 'the',
 'or',
 "mightn't",
 'themselves',
 's

In [4]:
import re
def removePunctuation(text):
    punctuation = '!,;:?"\'.-'
    text = re.sub(r'[{}]+'.format(punctuation),'',text)
    return text.strip().lower()
 

In [9]:
#loading corpus and preprocessing

corpus_list= []
for entry in entries:
    filepath = "/".join(["/content/drive/My Drive/words/sample", entry, "*.txt"])
    data_files = glob.glob(filepath)
    for data_file in data_files:
      document = open(data_file,'r', errors='ignore').read().lower()
      text = [word for word in document.split()] 
      corpus = [removePunctuation(word) for word in text]
      corpus = [word for word in corpus if not word in stop_words]
      corpus_list += [corpus]
len(corpus_list)





1324

In [10]:
#3 Merge corpus into one document
with open("/content/drive/My Drive/words/sample/BleiCorpus.txt", 'w') as File:
  for corpus in corpus_list:
    File.write(" ".join(corpus)+ "\n" )

In [12]:
## establish dictionary 
dictionary = corpora.Dictionary(corpus_list)
dictionary.save("/content/drive/My Drive/words/sample/word_dictionary")


## create vocabulary file
class MyCorpus(object):
    def __iter__(self):
        for line in open("/content/drive/My Drive/words/sample/BleiCorpus.txt"):
            yield dictionary.doc2bow(line.lower().split())

corpus_memory_friendly = MyCorpus()
corpus = [vector for vector in corpus_memory_friendly]
corpora.BleiCorpus.serialize('/content/drive/My Drive/words/sample/news_corpus', corpus)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [13]:
## loading and setting
try:
  dictionary = Dictionary.load("/content/drive/My Drive/words/sample/word_dictionary")
except FileNotFoundError as e:
  raise ValueError("SKIP: Please download the Corpus/news_dictionary dataset.")

corpus = bleicorpus.BleiCorpus("/content/drive/My Drive/words/sample/news_corpus")

time_slice = [438, 430, 456]


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [14]:
ldaseq = ldaseqmodel.LdaSeqModel(corpus=corpus, id2word=dictionary, time_slice=time_slice, num_topics=5)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
/usr/local/lib/python3.6/dist-packages/gensim/models/ldaseqmodel.py:290: RuntimeWarning: divide by zero encountered in double_scalars
  convergence = np.fabs((bound - old_bound) / old_bound)
/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function i

In [15]:
#Printing Topics
ldaseq.print_topics(time=0)

[[('people', 0.00874680075051746),
  ('new', 0.004843613657542955),
  ('one', 0.004695173880186267),
  ('users', 0.004492421322513193),
  ('use', 0.004324763984666068),
  ('technology', 0.004168938080818997),
  ('mobile', 0.0041139332500377025),
  ('software', 0.00383462493467108),
  ('games', 0.0034081757694272947),
  ('uk', 0.003337472410174551),
  ('many', 0.0031723099395692097),
  ('used', 0.003151857299842342),
  ('number', 0.0031305859608501845),
  ('net', 0.003080740745045981),
  ('online', 0.0029216809883559143),
  ('computer', 0.002825443290322916),
  ('phone', 0.0027995616757728737),
  ('digital', 0.002752542740046923),
  ('using', 0.0027441470230460427),
  ('internet', 0.0026996748967806697)],
 [('year', 0.006422498840852814),
  ('government', 0.004734466722997899),
  ('new', 0.004461117912408718),
  ('blair', 0.00421601194958282),
  ('labour', 0.0037225322596352835),
  ('election', 0.003710242104190938),
  ('last', 0.0033647985143102008),
  ('market', 0.003325824185144901),

In [16]:
#Topic Evolution
ldaseq.print_topic_times(topic=0)

[[('people', 0.00874680075051746),
  ('new', 0.004843613657542955),
  ('one', 0.004695173880186267),
  ('users', 0.004492421322513193),
  ('use', 0.004324763984666068),
  ('technology', 0.004168938080818997),
  ('mobile', 0.0041139332500377025),
  ('software', 0.00383462493467108),
  ('games', 0.0034081757694272947),
  ('uk', 0.003337472410174551),
  ('many', 0.0031723099395692097),
  ('used', 0.003151857299842342),
  ('number', 0.0031305859608501845),
  ('net', 0.003080740745045981),
  ('online', 0.0029216809883559143),
  ('computer', 0.002825443290322916),
  ('phone', 0.0027995616757728737),
  ('digital', 0.002752542740046923),
  ('using', 0.0027441470230460427),
  ('internet', 0.0026996748967806697)],
 [('people', 0.00886964763441495),
  ('new', 0.0048980166856164385),
  ('one', 0.004740918900276454),
  ('technology', 0.004256615158505419),
  ('use', 0.003938903251519493),
  ('users', 0.003847814662094136),
  ('mobile', 0.0037116511141609995),
  ('software', 0.0035848587783266594),


In [20]:
## word distribution in a specific document
words = [dictionary[word_id] for word_id, count in corpus[558]]
print (words)

['back', 'despite', 'earlier', 'hit', 'many', 'next', 'year', 'one', 'ask', 'court', 'december', 'group', 'real', 'month', 'done', 'president', 'record', 'work', 'brown', 'due', '50', 'promote', 'david', 'members', 'staff', 'suffered', 'arrested', 'away', 'charged', 'held', 'magazine', 'man', 'achievement', 'whose', 'attempted', 'holding', 'charge', 'second', 'receive', 'table', 'stage', 'counter', 'security', '20', 'name', 'involved', 'signed', 'award', 'stable', 'dr', 'young', 'james', 'awards', 'performer', 'ceremony', 'happened', 'condition', 'violence', 'appeared', 'music', 'allegedly', 'front', 'angeles', 'los', 'santa', 'stabbed', 'assault', 'murder', 'buck', 'attack', 'lifetime', 'dragged', 'label', 'rapper', 'crowd', 'johnson', 'jimmy', 'collapsed', 'hospital', 'cents', 'lung', 'dre', 'punching', 'dres', 'approached', '1000strong', 'autograph', 'darnell', 'deadly', 'ensuing', 'gibbs', 'gunit', 'hip', 'hop', 'kenard', 'monica', 'scuffle', 'seated', 'sickening', 'stabbing', 'vib

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [21]:
## A specific document's latent topic distribution
doc = ldaseq.doc_topics(558)
print(doc)

[2.25117166e-01 4.63142551e-02 8.32986256e-05 2.51892169e-01
 4.76593112e-01]


In [22]:
doc_football_1 = ['economy', 'bank', 'mobile', 'phone', 'markets', 'buy', 'football', 'united', 'giggs'] #make up a news about how football affect economy
doc_football_1 = dictionary.doc2bow(doc_football_1)
doc_football_1 = ldaseq[doc_football_1]
print (doc_football_1)

[0.2592062  0.40756043 0.00110497 0.33102343 0.00110497]


In [23]:
## topic distance between documents (hellinger distance: measure similarity of two distribution)
doc_football_2 = ['arsenal', 'fourth', 'wenger', 'oil', 'middle', 'east', 'sanction', 'fluctuation']
doc_football_2 = dictionary.doc2bow(doc_football_2)
doc_football_2 = ldaseq[doc_football_2]
hellinger(doc_football_1, doc_football_2)

0.3475448204093096

In [24]:
## changing rate of topic factors(words in topics)
ldaseq.print_topic_times(1)

##Note: the chain_variance parameter could affect the result

[[('year', 0.006422498840852814),
  ('government', 0.004734466722997899),
  ('new', 0.004461117912408718),
  ('blair', 0.00421601194958282),
  ('labour', 0.0037225322596352835),
  ('election', 0.003710242104190938),
  ('last', 0.0033647985143102008),
  ('market', 0.003325824185144901),
  ('growth', 0.0032055175836013783),
  ('economy', 0.00312463670033896),
  ('tax', 0.0030870979291680914),
  ('minister', 0.0030629642334538336),
  ('2004', 0.003007728188351138),
  ('sales', 0.0028215113874487793),
  ('economic', 0.002803639164053963),
  ('years', 0.0027590766714613592),
  ('brown', 0.0027457972770686834),
  ('next', 0.0026200635147074452),
  ('told', 0.0025734027317344714),
  ('prices', 0.0025461328528753675)],
 [('year', 0.006427926428040818),
  ('government', 0.004741485327876983),
  ('new', 0.004502122663777059),
  ('blair', 0.004282773504749348),
  ('labour', 0.003785514221338721),
  ('election', 0.0037842324279770572),
  ('last', 0.003372334646966216),
  ('market', 0.0033548062223